In [9]:
import numpy as np
import glob
from astropy.time import Time
import Optical_Processing_Functions as opf

# Process the optical observations of TYC 8332-2529-1

This Jupyter Notebook, and the python script <em>Process_Optical_Data.py</em>, processes the optical observations and write the data as the numpy files:
<ul>
    <li>TYC_optical_semesters.npy</li>
    <li>TYC_optical_binned.npy</li>
    <li>TYC_optical_binned_noOutliers.npy</li>
</ul>

The file <em>TYC_optical_semesters.npy</em> has the name, MJDs, magnitudes, and errors on the magnitudes of the ASAS, KELT, and ASAS-SN observations.

The file <em>TYC_optical_binned.npy</em> has the same
semesters, but binned such that there is only one MJD,
magntitude, and error per day (for each day that the
source is observed).

The file <em>TYC_optical_binned_noOutliers.npy</em> is
the same as <em>TYC_optical_binned.npy</em>, but the
outlier magnitudes have been removed.

The first step is the load in the KELT observations, and convert the times of the observations from Heliocentric Julian Date (HJD) to Modified Julian Date (MJD). KELT has East and West observations due to the instrument, you can find out more here: https://keltsurvey.org/lightcurves.

In [10]:
### The KELT data

east_tfa = np.genfromtxt('KELT_S36_lc_027057_V01_east_tfa.dat')
west_tfa = np.genfromtxt('KELT_S36_lc_027056_V01_west_tfa.dat')

east_tfa_time = east_tfa[:, 0]-2400000.5
east_tfa_V = east_tfa[:, 1] - 2.07
east_tfa_V_err = east_tfa[:, 2]

west_tfa_time = west_tfa[:, 0]-2400000.5
west_tfa_V = west_tfa[:, 1] - 2.07
west_tfa_V_err = west_tfa[:, 2]

Next, we load in the ASAS observations and process these. ASAS observations are graded A, B, C, or D. Where A observations are the highest quality, and D the lowest quality. Here we remove the C and D grade epochs, and also remove any remaining outliers (as these are due to instrumental issues).

In [11]:
### The ASAS data
asasfilename = 'ASAS_data.tsv'

(ASAS_times,
 ASAS_mags,
 ASAS_mag_errs) = opf.process_ASAS(asasfilename,
                                   good_ratings=True,
                                   cut_outliers=True)

Before processing:
Length times:  575 

After processing:
Length times:  534
Length data points:  534
Length uncertainties:  534
Faintest magnitude: 11.3386
Brightest magnitude: 10.6704


Finally, we load in the ASAS-SN observations. The ASAS-SN file has both g- and V-band observations in one file. So we split this data into the two separate bands.

In [12]:
### The ASAS-SN data
asassnfilename = 'ASASSN.csv'
(asassn_mjd_V,
 asassn_mag_V,
 asassn_mag_V_err,
 asassn_mjd_g,
 asassn_mag_g,
 asassn_mag_g_err) = opf.process_ASASSN(asassnfilename)

For the next step, put all the of the KELT, ASAS, and ASAS-SN data together.

In [13]:
### Turn all of the optical data into giant arrays
optical_names = ['ASAS', 'KELTs East', 'KELTs West',
                 'ASAS-SN V', 'ASAS-SN g']
optical_mjds = [ASAS_times, east_tfa_time, west_tfa_time,
                asassn_mjd_V, asassn_mjd_g]
optical_mags = [ASAS_mags, east_tfa_V, west_tfa_V,
                asassn_mag_V, asassn_mag_g]
optical_mag_errs = [ASAS_mag_errs, east_tfa_V_err, west_tfa_V_err,
                    asassn_mag_V_err, asassn_mag_g_err]

Now, we split the observations into semesters. A semester is the part of the year where the source is a night-time source (and as such, is visible to optical telescopes). Save these semesters in a numpy file so that they can be used for the Lomb-Scargle analysis and for plotting.

In [14]:
(optical_mjd_chunks,
 optical_mag_chunks,
 optical_mag_e_chunks,
 optical_name_chunks) = opf.split_semesters(optical_names, optical_mjds,
                                            optical_mags, optical_mag_errs,
                                            sufficient_points=20)
# Save these as a numpy file
np.save('TYC_optical_semesters', np.array([optical_mjd_chunks,
                                            optical_mag_chunks,
                                            optical_mag_e_chunks,
                                            optical_name_chunks]))

In plots the data looks messy, so bin the data. I bin it so that for each day the source isobserved, there's only one, average magnitude (and error).

I also make a version of this binned data where the extreme outliers are removed.

Save both of these binned data sets as numpy arrays for use in plotting.

In [15]:
(binned_times,
 binned_mags,
 binned_errs,
 no_times,
 no_mags,
 no_errs) = opf.bin_per_day(optical_mjd_chunks,
                            optical_mag_chunks,
                            optical_mag_e_chunks,
                            optical_name_chunks)
# Save these as numpy files
np.save('TYC_optical_binned', np.array([binned_times,
                                        binned_mags,
                                        binned_errs]))
np.save('TYC_optical_binned_noOutliers', np.array([no_times,
                                                   no_mags,
                                                   no_errs]))

So that you have an idea of the data, print out some basic information.

In [16]:
### Print out the basic info for each data set

total_days = 0
num_points = 0
for i, name in enumerate(optical_names):
    print('{0} Start MJD: {1:.2f} End MJD: {2:.2f}'.format(name,
                                                           np.min(optical_mjds[i]),
                                                           np.max(optical_mjds[i])))
    print('Number of data points: {}'.format(len(optical_mjds[i])))
    print('Start ISOT: ', Time(np.min(optical_mjds[i]), format='mjd').isot)
    print('End ISOT', Time(np.max(optical_mjds[i]), format='mjd').isot)
    days = (Time(np.max(optical_mjds[i]),
                 format='mjd')-
            Time(np.min(optical_mjds[i]),
                 format='mjd'))
    print('Total number of days: {0}'.format(int((days.sec)/60./60./24.)))
    print()

    num_points += len(optical_mjds[i])
    total_days += (days.sec)/60./60./24.

print('Total number of days of observations: {}'.format(int(total_days)))
print('Total number of data points: {}'.format(num_points))
print('Total span of observations: {:.2f} years'.format((Time(np.max(optical_mjds[-1]),
                                                          format='mjd')-
                                                     Time(np.min(optical_mjds[0]),
                                                          format='mjd')).sec/60./60./24./365.25))

ASAS Start MJD: 51933.38 End MJD: 55116.02
Number of data points: 534
Start ISOT:  2001-01-24T09:00:21.600
End ISOT 2009-10-12T00:27:05.184
Total number of days: 3182

KELTs East Start MJD: 56427.89 End MJD: 57254.75
Number of data points: 1425
Start ISOT:  2013-05-15T21:21:49.738
End ISOT 2015-08-20T18:03:10.166
Total number of days: 826

KELTs West Start MJD: 56428.03 End MJD: 57308.77
Number of data points: 1612
Start ISOT:  2013-05-16T00:42:12.384
End ISOT 2015-10-13T18:34:12.691
Total number of days: 880

ASAS-SN V Start MJD: 57457.35 End MJD: 58383.03
Number of data points: 759
Start ISOT:  2016-03-10T08:27:09.216
End ISOT 2018-09-22T00:37:04.800
Total number of days: 925

ASAS-SN g Start MJD: 58175.39 End MJD: 58700.22
Number of data points: 645
Start ISOT:  2018-02-26T09:21:11.808
End ISOT 2019-08-05T05:22:14.592
Total number of days: 524

Total number of days of observations: 6340
Total number of data points: 4975
Total span of observations: 18.53 years
